In [19]:
# webscrape 20 images each of 10 anime characters
# save images to folder

import os
from bing_image_downloader import downloader

# create folder to save images if it doesn't exist
if not os.path.exists('images'):
    os.mkdir('images')

# create list of anime characters
anime_characters = ['saitama', 'goku', 'naruto', 'luffy', 'light', 'sasuke', 'killua', 'gintoki', 'saitama', 'goku']

#delete all folders which are not in anime_characters
for folder in os.listdir('images'):
    if folder not in anime_characters:
        os.rmdir('images/' + folder)

In [21]:
# loop through each character
for character in anime_characters:
    # download 20 images of each character
    downloader.download(character+" face", limit=10, output_dir='images/', adult_filter_off=True, force_replace=False, timeout=60, verbose=False, filter="photo")

[%] Downloading Images to d:\DSLab\ML\EigenFaces\images\saitama face
[!] Issue getting: https://external-preview.redd.it/UhajG7Z8o_YK30AIvVpVGWnTTteZ0htAJ82yQD-fSb8.png?auto=webp&amp;s=a5de2cb6c9a982755a7ca8bf5f5af8c505198553
[!] Error:: HTTP Error 403: Forbidden


[%] Done. Downloaded 10 images.
[%] Downloading Images to d:\DSLab\ML\EigenFaces\images\goku face


[%] Done. Downloaded 10 images.
[%] Downloading Images to d:\DSLab\ML\EigenFaces\images\naruto face


[%] Done. Downloaded 10 images.
[%] Downloading Images to d:\DSLab\ML\EigenFaces\images\luffy face
[Error]Invalid image, not saving http://images5.fanpop.com/image/photos/29900000/Luffy-one-piece-29912041-960-720.jpg

[!] Issue getting: http://images5.fanpop.com/image/photos/29900000/Luffy-one-piece-29912041-960-720.jpg
[!] Error:: Invalid image, not saving http://images5.fanpop.com/image/photos/29900000/Luffy-one-piece-29912041-960-720.jpg



[%] Done. Downloaded 10 images.
[%] Downloading Images to d:\DSLab\ML\EigenFaces\ima

KeyboardInterrupt: 